# ATTENTION: PLEASE SEE THE README FILE AND CREATE A CONDA ENVIRONMENT WITH THE GIVEN .yml FILE, USE THAT ENVIRONMENT TO RUN THIS SCRIPT.

# Libraries (only needed in Kaggle environment) and Dataset download 

In [ ]:
# !pip install mne
!kaggle datasets download -d huephamkim/brain-age -p data/brain_age

# Imports

In [ ]:
import gc
import os
import zipfile

import kagglehub
import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import psutil
import seaborn as sns
import tensorflow as tf
from joblib import Parallel, delayed
from scipy.signal import welch
from scipy.stats import kurtosis, skew
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (BatchNormalization, Bidirectional, Conv2D, Dense, 
                                     Dropout, Flatten, LSTM, MaxPooling2D, Reshape, 
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tqdm.auto import tqdm


# Data download

In [ ]:
# Path to the ZIP file
zip_file_path = 'data/brain_age/brain-age.zip'

# Destination folder
extract_to_path = 'data/brain_age'

# Create the folder if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print(f"Dataset extracted to: {extract_to_path}")


# Plotting the age distribution of the data

In [ ]:
file_path = 'data/brain_age/filtered_subjects_with_age.tsv' 
label = pd.read_csv(file_path, sep='\t')

label

In [ ]:
# Extract the values of the 'participant_id' column from the label
subjects=label['participant_id'].values
filtered_data = label[label['participant_id'].isin(subjects)]

# Displays a histogram showing the age frequency of the data
plt.figure(figsize=(10, 6))
sns.histplot(filtered_data['age'], bins=25, kde=True, color='red', label='Age Distribution')
plt.title(f'Age Distribution of {160} Subjects', fontsize=16)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Data Preprocessing functions

In [58]:
def load_and_split_data(labels_df, data_dir):
    """
    Loads and preprocesses EEG data for each subject listed in the given labels dataframe.

    Parameters:
    ----------
    labels_df : pd.DataFrame
        Dataframe containing participant IDs and their corresponding ages.
    data_dir : str
        The directory containing EEG data files. Each file is named as 
        '{participant_id}_sflip_parc-raw.fif'.

    Returns:
    -------
    tuple
        A tuple containing:
        - datas (numpy.ndarray): A 3D array of shape (n_samples, n_channels, n_timepoints),
          representing the processed EEG data for all subjects and epochs.
        - labels (numpy.ndarray): A 1D array of shape (n_samples,) containing the age labels
          corresponding to each sample in the `datas` array.

    Raises:
    ------
    ValueError
        If no valid data is found after preprocessing, indicating that the provided data
        did not meet the preprocessing requirements.

    Notes:
    ------
    - This function expects EEG data in FIF format, with misc channels available.
    - The preprocessing includes:
        1. Selecting only misc channels.
        2. Resampling the data to 250 Hz.
        3. Creating fixed-length epochs of 6 seconds each.
    - Each epoch is associated with the age of the subject from the TSV file.
    """
    id_to_age = dict(zip(labels_df['participant_id'], labels_df['age']))
    subjects = labels_df['participant_id'].values
    
    datas = []
    labels = []
    
    for subject in tqdm(subjects):
        path = f"{data_dir}/{subject}_sflip_parc-raw.fif"
        raw = mne.io.read_raw_fif(path, preload=True, verbose=False)
        
        # Get misc channel indices
        misc_picks = mne.pick_types(raw.info, misc=True)
        if len(misc_picks) == 0:
            print(f"Warning: No misc channels found for subject {subject}")
            continue
            
        # Apply preprocessing
        raw.pick_types(misc=True)
        raw.resample(250)
        
        epochs = mne.make_fixed_length_epochs(raw, duration=6)
        data = epochs.get_data()
        
        for epoch_data in data:
            datas.append(epoch_data)
            labels.append(id_to_age[subject])
    
    if len(datas) == 0:
        raise ValueError("No valid data found after preprocessing")
        
    return np.array(datas), np.array(labels)

In [59]:
def load_and_preprocess_data(labels_df, data_dir):
    """
    Loads and preprocesses EEG data for each subject listed in the given labels dataframe.
    
    Parameters:
    ----------
    labels_df : pd.DataFrame
        Dataframe containing participant IDs and their corresponding ages.
    data_dir : str
        The directory containing EEG data files. Each file is named as 
        '{participant_id}_sflip_parc-raw.fif'.
        
    Returns:
    -------
    tuple
        A tuple containing:
        - datas (numpy.ndarray): A 3D array of shape (n_samples, n_channels, n_timepoints),
          representing the processed EEG data for all subjects and epochs.
        - labels (numpy.ndarray): A 1D array of shape (n_samples,) containing the age labels
          corresponding to each sample in the `datas` array.

    
    Raises:
    ------
    ValueError
        If no valid data is found after preprocessing, indicating that the provided data
        did not meet the preprocessing requirements.
    
    Notes:
    ------
    - This function expects EEG data in FIF format, with misc channels available.
    - The preprocessing includes:
        1. Selecting only misc channels.
        2. Resampling the data to 250 Hz.
        3. The data is filtered with a bandpass filter between 1 Hz and 100 Hz
        4. Creating fixed-length epochs of 6 seconds each.
    - Each epoch is associated with the age of the subject from the TSV file.
    """
    id_to_age = dict(zip(labels_df['participant_id'], labels_df['age']))
    subjects = labels_df['participant_id'].values
    
    datas = []
    labels = []
    
    for subject in tqdm(subjects):
        path = f"{data_dir}/{subject}_sflip_parc-raw.fif"
        raw = mne.io.read_raw_fif(path, preload=True, verbose=False)
        
        # Get misc channel indices
        misc_picks = mne.pick_types(raw.info, misc=True)
        if len(misc_picks) == 0:
            print(f"Warning: No misc channels found for subject {subject}")
            continue
            
        # Apply preprocessing
        raw.pick_types(misc=True)
        raw.resample(250)
        raw.filter(l_freq=1, h_freq=100, picks='misc')
        
        epochs = mne.make_fixed_length_epochs(raw, duration=6)
        data = epochs.get_data()
        
        for epoch_data in data:
            datas.append(epoch_data)
            labels.append(id_to_age[subject])
    
    if len(datas) == 0:
        raise ValueError("No valid data found after preprocessing")
        
    return np.array(datas), np.array(labels)

In [60]:
def calc_Features(signal):
    """
    Extracts a variety of time-domain, frequency-domain, and complexity features from the given signal.
    
    This function calculates multiple statistical and spectral features that describe the signal's properties.
    It includes both time-domain features (such as mean, variance, kurtosis) and frequency-domain features (such as
    spectral power and peak frequency). Additionally, it computes a simple complexity measure.

    Parameters:
    ----------
    signal: array-like
        The input signal (e.g., EEG signal) for which the features will be extracted. 
        The signal should be a 1D array or list of numerical values.

    Returns:
    -------
    np.ndarray: A numpy array containing the extracted features. The array includes time-domain features, 
        frequency-domain features, and complexity measures in the following order:
        - Time-domain features: mean, standard deviation, variance, kurtosis, skewness, max, min,
          mean absolute value, and median absolute value.
        - Frequency-domain features: standard deviation, max, peak frequency, and spectral centroid.
        - Complexity measures: mean absolute difference.

    Notes:
    ------
    - The frequency-domain features are computed using the Power Spectral Density (PSD) estimate obtained via 
      the Welch method. The sampling frequency is assumed to be 250 Hz.
    - The time-domain features are derived from basic statistical operations on the signal.
    - The function assumes that the input signal is 1D, and any multi-dimensional signal should be flattened
      prior to calling the function.
    """
    # Convert the signal to a numpy array
    signal = np.array(signal)
    features = []
    
    # Time domain features
    features.extend([
        np.mean(signal),  # Mean
        np.std(signal),   # Standard deviation
        np.var(signal),   # Variance
        kurtosis(signal), # Kurtosis
        skew(signal),     # Skewness
        np.max(signal),   # Maximum value
        np.min(signal),   # Minimum value
        np.mean(np.abs(signal)),  # Mean of the absolute values
        np.median(np.abs(signal)) # Median of the absolute values
    ])
    
    # Frequency domain features
    freqs, psd = welch(signal, fs=250, nperseg=min(len(signal), 256))  # Calculate PSD
    features.extend([
        np.std(psd),                # Standard deviation of the power spectral density
        np.max(psd),                # Maximum value in the power spectral density
        freqs[np.argmax(psd)],      # Peak frequency (frequency corresponding to max PSD)
        np.mean(freqs * psd) / np.mean(psd)  # Spectral centroid (weighted average frequency)
    ])
    
    # Complexity measures
    features.extend([
        np.mean(np.abs(np.diff(signal))),  # Mean absolute difference consecutive samples
    ])
    
    return np.array(features)

In [61]:
def extract_features_parallel(X):
    """
    Extracts features from a multidimensional signal array in parallel across samples and channels.
    
    This function utilizes parallel processing to extract features from each channel of each sample in the input
    array `X` by applying the `calc_Features` function. The computation is parallelized to speed up the extraction
    process, especially useful for large datasets. The function returns a 3D array where each entry contains the 
    features extracted from a specific channel of a sample.

    Parameters:
    ----------
    X: np.ndarray
        A 3D numpy array representing the input signals with the shape (n_samples, n_channels, n_time_points),
        where:
        - `n_samples` is the number of samples (e.g., data points, subjects, or recordings).
        - `n_channels` is the number of channels (e.g., EEG channels or sensor data).
        - `n_time_points` is the length of the signal for each channel (e.g., the number of time samples or measurements).

    Returns:
    -------
    np.ndarray: A 3D numpy array of extracted features with the shape (n_samples, n_channels, n_features),
        where `n_features` is the number of features extracted for each channel from each sample. These features
        are derived by applying the `calc_Features` function to each channel of each sample.

    Notes:
    ------
    - The function assumes that the `calc_Features` function can extract a fixed set of features from each individual signal.
    - Parallel processing is used through `joblib.Parallel` to speed up the computation by utilizing multiple CPU cores.
    - The output shape will be `(n_samples, n_channels, n_features)`, where `n_features` is the number of features returned by `calc_Features`.
    """
    n_samples, n_channels, _ = X.shape
    features_list = Parallel(n_jobs=-1)(
        delayed(calc_Features)(X[i, j]) 
        for i in tqdm(range(n_samples)) 
        for j in range(n_channels)
    )
    return np.array(features_list).reshape(n_samples, n_channels, -1)

# Splitting data

In [ ]:
train_labels, test_labels = train_test_split(label, test_size=40, random_state=42)

train_labels.shape, test_labels.shape

# Models

## CNN-BiLSTM

In [12]:
def CNN_BiLSTM(input_shape):
    """
    Create a CNN + BiLSTM model for sequential data processing and prediction.

    This function defines a deep learning model combining Convolutional Neural Networks (CNN) 
    for feature extraction and Bidirectional Long Short-Term Memory (BiLSTM) networks for 
    sequential data modeling.

    Parameters:
        input_shape (tuple): The shape of the input data (height, width, channels).

    Returns:
        model (Sequential): A compiled Keras Sequential model.

    Model Architecture:
        1. Convolutional Layers:
            - Two Conv2D layers with ReLU activation for spatial feature extraction.
            - BatchNormalization for stabilizing and accelerating training.
            - MaxPooling2D for dimensionality reduction.
            - Dropout to reduce overfitting.

        2. Global Pooling:
            - TimeDistributed Flatten layer for reducing dimensionality while preserving temporal order.

        3. LSTM Layers:
            - Bidirectional LSTM layer for capturing temporal dependencies in both directions.
            - Additional LSTM layer for sequential processing.
            - Dropout layers for regularization.

        4. Fully Connected Layers:
            - Dense layers for further feature abstraction.
            - Final Dense layer with a linear activation for regression output.
    """
    model = Sequential([
        # Convolutional Layers
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),
        
        # Global Pooling for dimensionality reduction
        TimeDistributed(Flatten()),
        
        # LSTM Layers
        Bidirectional(LSTM(32, return_sequences=True)),
        Dropout(0.3),
        LSTM(20, return_sequences=False),
        Dropout(0.3),
        
        # Fully Connected Layers
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='linear')
    ])
    
    return model

### Approach 1: Training the model with raw data (no preprocessing)

**Loading the data withous processing**

In [ ]:
# Set the logging level for MNE to 'ERROR'
mne.set_log_level('ERROR')

X_train, y_train = load_and_split_data(
        train_labels,
        "data/brain_age"
    )

X_test, y_test = load_and_split_data(
        test_labels,
        "data/brain_age"
    )

X_train.shape, X_test.shape

In [14]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

In [ ]:
y_train, y_test

In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

X_train.shape, X_test.shape

**Model configs**

In [ ]:
model_1 = CNN_BiLSTM((X_train.shape[1], X_train.shape[2], X_train.shape[3]))

# Compile the model
model_1.compile(
    optimizer='adam', 
    loss='mse', 
    metrics=['mae']
)

# Summary
model_1.summary()

In [18]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, 
    restore_best_weights=True  # Restore the weights of the best epoch
)

**Training**

In [ ]:
# training 
history = model_1.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=200, 
    batch_size=32, 
    callbacks=[early_stopping]
)

**Evaluation**

In [ ]:
loss, mae = model_1.evaluate(X_test, y_test)
print(f"Test Loss (MSE): {loss}, Test MAE: {mae}")

In [ ]:
y_pred = model_1.predict(X_test[::50])  # Model outputs predictions

# Print predictions vs. real labels
for i, real_label in enumerate(y_test[::50]):  # Use enumerate to get the index
    print(f"Sample {i+1}: Prediction = {y_pred[i][0]:.2f}, Real Label = {real_label:.2f}")

**Plotting learning curve**

In [ ]:
# Extract loss and MAE data from the training history
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']

# Plot the loss and MAE graphs
plt.figure(figsize=(12, 6))

# Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(loss) + 1), loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# MAE
plt.subplot(1, 2, 2)
plt.plot(range(1, len(mae) + 1), mae, label='Training MAE')
plt.plot(range(1, len(val_mae) + 1), val_mae, label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('MAE over Epochs')
plt.legend()

# Display
plt.tight_layout()
plt.show()

**Deleting variables to save on RAM usage (Deletion for resources saving will be performed several times in this script)**

In [23]:
del X_train
del X_test
del y_train
del y_test
del model_1

In [ ]:
gc.collect()

In [ ]:
# Memory usage in MB
process = psutil.Process()
print(f"Current memory usage: {process.memory_info().rss / (1024**2):.2f} MB")

### Approach 2: Training model with processed data

In [ ]:
mne.set_log_level('ERROR')

X_train, y_train = load_and_preprocess_data(
        train_labels,
        "data/brain_age"
    )

X_test, y_test = load_and_preprocess_data(
        test_labels,
        "data/brain_age"
    )

X_train.shape, X_test.shape

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

y_train, y_test

In [ ]:
print("Extracting features...")
X_train_features = extract_features_parallel(X_train)
X_test_features = extract_features_parallel(X_test)

In [29]:
del X_train
del X_test

**Scaling data**

In [ ]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(
    X_train_features.reshape(X_train_features.shape[0], -1)
)
X_test_scaled = scaler.transform(
    X_test_features.reshape(X_test_features.shape[0], -1)
)

X_train_reshaped = X_train_scaled.reshape(
    X_train_scaled.shape[0], 
    X_train_features.shape[1], 
    X_train_features.shape[2]
)
X_test_reshaped = X_test_scaled.reshape(
    X_test_scaled.shape[0], 
    X_test_features.shape[1], 
    X_test_features.shape[2]
)

X_train_reshaped.shape, X_test_reshaped.shape

In [31]:
del X_train_features
del X_test_features
del X_train_scaled
del X_test_scaled

In [ ]:
X_train_reshaped = np.expand_dims(X_train_reshaped, axis=-1)
X_test_reshaped = np.expand_dims(X_test_reshaped, axis=-1)

X_train_reshaped.shape, X_test_reshaped.shape

**Model configs**

In [ ]:
model_2 = CNN_BiLSTM((X_train_reshaped.shape[1], X_train_reshaped.shape[2], X_train_reshaped.shape[3]))

# Compile the model
model_2.compile(
    optimizer='adam', 
    loss='mse', 
    metrics=['mae']
)

# Summary
model_2.summary()

In [34]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10, 
    restore_best_weights=True  # Restore the weights of the best epoch
)

**Training**

In [ ]:
history = model_2.fit(
    X_train_reshaped, y_train, 
    validation_data=(X_test_reshaped, y_test), 
    epochs=200, 
    batch_size=32, 
    callbacks=[early_stopping]
)

**Evaluation**

In [ ]:
loss, mae = model_2.evaluate(X_test_reshaped, y_test)
print(f"Test Loss (MSE): {loss}, Test MAE: {mae}")

In [ ]:
y_pred = model_2.predict(X_test_reshaped[::50])  # Model outputs predictions

# Print predictions vs. real labels
for i, real_label in enumerate(y_test[::50]):  # Use enumerate to get the index
    print(f"Sample {i+1}: Prediction = {y_pred[i][0]:.2f}, Real Label = {real_label:.2f}")

**Plotting learning curve**

In [ ]:
# Extract loss and MAE data from the training history
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']

# Plot the loss and MAE graphs
plt.figure(figsize=(12, 6))

# Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(loss) + 1), loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# MAE
plt.subplot(1, 2, 2)
plt.plot(range(1, len(mae) + 1), mae, label='Training MAE')
plt.plot(range(1, len(val_mae) + 1), val_mae, label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('MAE over Epochs')
plt.legend()

# Display
plt.tight_layout()
plt.show()

In [39]:
del model_2

In [ ]:
X_train_reshaped = np.squeeze(X_train_reshaped, axis=-1)
X_test_reshaped = np.squeeze(X_test_reshaped, axis=-1)

X_train_reshaped.shape, X_test_reshaped.shape

## LSTM

### This model is trained on preprocessed data

In [41]:
def create_lstm_model(input_shape):
    """
    Creates an LSTM-based neural network model for time series or sequential data prediction tasks.
    
    The model consists of multiple LSTM layers for sequence processing, 
    followed by fully connected layers for feature extraction and output prediction. 
    Batch normalization and dropout layers are used for regularization to improve 
    model generalization and prevent overfitting.
    
    Parameters:
    ----------
    input_shape : tuple
        The shape of the input data, excluding the batch size. For example, 
        for a dataset with 100 timesteps and 10 features, input_shape should be (100, 10).

    Returns:
    -------
    model : keras.Sequential
        A compiled Keras Sequential model with LSTM layers and fully connected layers.
    
    Model Architecture:
    -------------------
    1. Input Layer: Batch Normalization to normalize the input data.
    2. LSTM Layer 1: 256 units with return_sequences=True to output sequences for the next layer.
       - Batch Normalization
       - Dropout (rate: 0.4)
    3. LSTM Layer 2: 128 units with return_sequences=True.
       - Batch Normalization
       - Dropout (rate: 0.4)
    4. LSTM Layer 3: 64 units (final LSTM layer).
       - Batch Normalization
       - Dropout (rate: 0.3)
    5. Dense Layer 1: Fully connected layer with 32 neurons and ReLU activation.
       - Batch Normalization
       - Dropout (rate: 0.2)
    6. Dense Layer 2: Fully connected layer with 16 neurons and ReLU activation.
       - Batch Normalization
    7. Output Layer: Fully connected layer with 1 neuron and linear activation for regression output.
    
    Notes:
    ------
    - Ensure the input data shape matches the `input_shape` specified.
    - The model's final output is designed for regression tasks. For classification, 
      modify the output layer activation and loss function accordingly.
    """
    model = Sequential([
        BatchNormalization(input_shape=input_shape),
        LSTM(256, return_sequences=True),
        BatchNormalization(),
        Dropout(0.4),
        
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.4),
        
        LSTM(64),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(32, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(16, activation='relu'),
        BatchNormalization(),
        
        Dense(1, activation='linear')
    ])
    
    return model

**Model configs**

In [ ]:
model_3 = create_lstm_model((X_train_reshaped.shape[1], X_train_reshaped.shape[2]))

model_3.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse', 
    metrics=['mae']
)

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_mae',
        patience=15,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_mae',
        factor=0.5,
        patience=7,
        min_lr=1e-6
    )
]

model_3.summary()

**Training**

In [ ]:
# Training
history = model_3.fit(
    X_train_reshaped, y_train,
    validation_data=(X_test_reshaped, y_test),
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

**Evaluating**

In [ ]:
loss, mae = model_3.evaluate(X_test_reshaped, y_test)
print(f"Test Loss (MSE): {loss}, Test MAE: {mae}")

In [ ]:
# Make predictions
y_pred = model_3.predict(X_test_reshaped[::50])  # Model outputs predictions

# Print predictions vs. real labels
for i, real_label in enumerate(y_test[::50]):  # Use enumerate to get the index
    print(f"Sample {i+1}: Prediction = {y_pred[i][0]:.2f}, Real Label = {real_label:.2f}")

**Plotting learning curve**

In [ ]:
# Extract loss and MAE data from the training history
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']

# Plot the loss and MAE graphs
plt.figure(figsize=(12, 6))

# Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(loss) + 1), loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# MAE
plt.subplot(1, 2, 2)
plt.plot(range(1, len(mae) + 1), mae, label='Training MAE')
plt.plot(range(1, len(val_mae) + 1), val_mae, label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('MAE over Epochs')
plt.legend()

# Display
plt.tight_layout()
plt.show()

# Machine learning models

In [ ]:
# Flatten the feature dimensions
X_train_flat = X_train_reshaped.reshape(X_train_reshaped.shape[0], -1)
X_test_flat = X_test_reshaped.reshape(X_test_reshaped.shape[0], -1)

# Check the shapes after flattening
print(X_train_flat.shape, X_test_flat.shape)  # Should be (n_samples, n_flat_features)

## RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Train the RandomForest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_flat, y_train)

# Make predictions
y_pred_train = rf_model.predict(X_train_flat)
y_pred_test = rf_model.predict(X_test_flat)

# Evaluate the model
train_mae = mean_absolute_error(y_train, y_pred_train)
train_mse = mean_squared_error(y_train, y_pred_train)

test_mae = mean_absolute_error(y_test, y_pred_test)
test_mse = mean_squared_error(y_test, y_pred_test)

print(f"Train MAE: {train_mae}")
print(f"Test MAE: {test_mae}")

## XGBRegressor

In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train_flat, y_train)
y_pred_test_xgb = xgb_model.predict(X_test_flat)

test_mae_xgb = mean_absolute_error(y_test, y_pred_test_xgb)
y_pred_train_xgb = xgb_model.predict(X_train_flat)

# Calculate Train MAE
train_mae_xgb = mean_absolute_error(y_train, y_pred_train_xgb)

# Print Train and Test MAE
print(f"XGBoost Train MAE: {train_mae_xgb}")
print(f"XGBoost Test MAE: {test_mae_xgb}")

## LGBMRegressor

In [ ]:
from lightgbm import LGBMRegressor

lgbm_model = LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
lgbm_model.fit(X_train_flat, y_train)
y_pred_lgbm = lgbm_model.predict(X_test_flat)

test_mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
y_pred_train_lgbm = lgbm_model.predict(X_train_flat)

# Calculate Train MAE
train_mae_lgbm = mean_absolute_error(y_train, y_pred_train_lgbm)

# Print Train and Test MAE
print(f"LightGBM Train MAE: {train_mae_xgb}")
print(f"LightGBM Test MAE: {test_mae_lgbm}")

In [ ]:
# Make predictions
y_pred = rf_model.predict(X_test_flat[:50])  # Predict on a subset of test data

# Print predictions vs. real labels
for i in range(len(y_test[:50])):
    print(f"Sample {i+1}: Prediction = {y_pred[i]:.2f}, Real Label = {y_test[i]:.2f}")

### Bonus visualizations: Correlation map between extracted features from data processing

In [ ]:
reshaped_data = X_train_reshaped[0]

# Correlation matrix between features
correlation_matrix = np.corrcoef(reshaped_data, rowvar=False)

# Plot matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix between Features')
plt.show()